# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f350cfbe310>
├── 'a' --> tensor([[-1.3875, -1.3430,  1.2040],
│                   [ 0.0186,  0.7092,  0.1782]])
└── 'x' --> <FastTreeValue 0x7f350cfbe3d0>
    └── 'c' --> tensor([[ 0.6477, -0.4423,  0.8207, -1.5547],
                        [-0.0033, -0.2649, -0.7218,  0.1888],
                        [ 1.3174,  1.4163,  0.4638, -1.0683]])

In [4]:
t.a

tensor([[-1.3875, -1.3430,  1.2040],
        [ 0.0186,  0.7092,  0.1782]])

In [5]:
%timeit t.a

62.3 ns ± 0.0207 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f350cfbe310>
├── 'a' --> tensor([[-0.4756, -0.6189,  0.0260],
│                   [-0.8933, -0.1587,  0.2385]])
└── 'x' --> <FastTreeValue 0x7f350cfbe3d0>
    └── 'c' --> tensor([[ 0.6477, -0.4423,  0.8207, -1.5547],
                        [-0.0033, -0.2649, -0.7218,  0.1888],
                        [ 1.3174,  1.4163,  0.4638, -1.0683]])

In [7]:
%timeit t.a = new_value

69.5 ns ± 0.0287 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.3875, -1.3430,  1.2040],
               [ 0.0186,  0.7092,  0.1782]]),
    x: Batch(
           c: tensor([[ 0.6477, -0.4423,  0.8207, -1.5547],
                      [-0.0033, -0.2649, -0.7218,  0.1888],
                      [ 1.3174,  1.4163,  0.4638, -1.0683]]),
       ),
)

In [10]:
b.a

tensor([[-1.3875, -1.3430,  1.2040],
        [ 0.0186,  0.7092,  0.1782]])

In [11]:
%timeit b.a

59.4 ns ± 0.581 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.5158,  0.5694,  1.9901],
               [-0.5177,  0.8187, -0.6208]]),
    x: Batch(
           c: tensor([[ 0.6477, -0.4423,  0.8207, -1.5547],
                      [-0.0033, -0.2649, -0.7218,  0.1888],
                      [ 1.3174,  1.4163,  0.4638, -1.0683]]),
       ),
)

In [13]:
%timeit b.a = new_value

489 ns ± 0.142 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

860 ns ± 3.76 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 12.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

140 µs ± 1.9 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 259 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f3467d06cd0>
├── 'a' --> tensor([[[-1.3875, -1.3430,  1.2040],
│                    [ 0.0186,  0.7092,  0.1782]],
│           
│                   [[-1.3875, -1.3430,  1.2040],
│                    [ 0.0186,  0.7092,  0.1782]],
│           
│                   [[-1.3875, -1.3430,  1.2040],
│                    [ 0.0186,  0.7092,  0.1782]],
│           
│                   [[-1.3875, -1.3430,  1.2040],
│                    [ 0.0186,  0.7092,  0.1782]],
│           
│                   [[-1.3875, -1.3430,  1.2040],
│                    [ 0.0186,  0.7092,  0.1782]],
│           
│                   [[-1.3875, -1.3430,  1.2040],
│                    [ 0.0186,  0.7092,  0.1782]],
│           
│                   [[-1.3875, -1.3430,  1.2040],
│                    [ 0.0186,  0.7092,  0.1782]],
│           
│                   [[-1.3875, -1.3430,  1.2040],
│                    [ 0.0186,  0.7092,  0.1782]]])
└── 'x' --> <FastTreeValue 0x7f346e91ffa0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.4 µs ± 86.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f351404c2b0>
├── 'a' --> tensor([[-1.3875, -1.3430,  1.2040],
│                   [ 0.0186,  0.7092,  0.1782],
│                   [-1.3875, -1.3430,  1.2040],
│                   [ 0.0186,  0.7092,  0.1782],
│                   [-1.3875, -1.3430,  1.2040],
│                   [ 0.0186,  0.7092,  0.1782],
│                   [-1.3875, -1.3430,  1.2040],
│                   [ 0.0186,  0.7092,  0.1782],
│                   [-1.3875, -1.3430,  1.2040],
│                   [ 0.0186,  0.7092,  0.1782],
│                   [-1.3875, -1.3430,  1.2040],
│                   [ 0.0186,  0.7092,  0.1782],
│                   [-1.3875, -1.3430,  1.2040],
│                   [ 0.0186,  0.7092,  0.1782],
│                   [-1.3875, -1.3430,  1.2040],
│                   [ 0.0186,  0.7092,  0.1782]])
└── 'x' --> <FastTreeValue 0x7f346ebf3af0>
    └── 'c' --> tensor([[ 0.6477, -0.4423,  0.8207, -1.5547],
                        [-0.0033, -0.2649, -0.7218,  0.1888],
                 

In [23]:
%timeit t_cat(trees)

31.1 µs ± 105 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.2 µs ± 81.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.6477, -0.4423,  0.8207, -1.5547],
                       [-0.0033, -0.2649, -0.7218,  0.1888],
                       [ 1.3174,  1.4163,  0.4638, -1.0683]],
              
                      [[ 0.6477, -0.4423,  0.8207, -1.5547],
                       [-0.0033, -0.2649, -0.7218,  0.1888],
                       [ 1.3174,  1.4163,  0.4638, -1.0683]],
              
                      [[ 0.6477, -0.4423,  0.8207, -1.5547],
                       [-0.0033, -0.2649, -0.7218,  0.1888],
                       [ 1.3174,  1.4163,  0.4638, -1.0683]],
              
                      [[ 0.6477, -0.4423,  0.8207, -1.5547],
                       [-0.0033, -0.2649, -0.7218,  0.1888],
                       [ 1.3174,  1.4163,  0.4638, -1.0683]],
              
                      [[ 0.6477, -0.4423,  0.8207, -1.5547],
                       [-0.0033, -0.2649, -0.7218,  0.1888],
                       [ 1.3174,  1.4163,  0.4638, -1.0683]],

In [26]:
%timeit Batch.stack(batches)

78.3 µs ± 98.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.6477, -0.4423,  0.8207, -1.5547],
                      [-0.0033, -0.2649, -0.7218,  0.1888],
                      [ 1.3174,  1.4163,  0.4638, -1.0683],
                      [ 0.6477, -0.4423,  0.8207, -1.5547],
                      [-0.0033, -0.2649, -0.7218,  0.1888],
                      [ 1.3174,  1.4163,  0.4638, -1.0683],
                      [ 0.6477, -0.4423,  0.8207, -1.5547],
                      [-0.0033, -0.2649, -0.7218,  0.1888],
                      [ 1.3174,  1.4163,  0.4638, -1.0683],
                      [ 0.6477, -0.4423,  0.8207, -1.5547],
                      [-0.0033, -0.2649, -0.7218,  0.1888],
                      [ 1.3174,  1.4163,  0.4638, -1.0683],
                      [ 0.6477, -0.4423,  0.8207, -1.5547],
                      [-0.0033, -0.2649, -0.7218,  0.1888],
                      [ 1.3174,  1.4163,  0.4638, -1.0683],
                      [ 0.6477, -0.4423,  0.8207, -1.5547],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 214 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

312 µs ± 1.65 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
